In [ ]:
import itertools
import random

import torch
from torch.nn import Linear
from torch.nn import functional as F
from torch.nn.functional import cosine_similarity
from torch.optim import Adam
from torch_geometric.data import Data
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import BatchNorm, MessagePassing, TopKPooling
from torch_geometric.transforms import NormalizeFeatures
from torch_scatter import scatter_mean
from collections import defaultdict

from custom.args import grey, purple
from custom.dataset import GraphDataset, create_dataset
from custom.utils import *
from networkx.algorithms.centrality import degree_centrality
import pickle
from itertools import zip_longest
from torch_geometric.utils import degree

In [ ]:
class GraphMatchingConvolution(MessagePassing):
    def __init__(self, in_channels, out_channels, args, aggr="add"):
        super(GraphMatchingConvolution, self).__init__(aggr=aggr)
        self.args = args
        self.lin_node = torch.nn.Linear(in_channels, out_channels)
        self.lin_message = torch.nn.Linear(out_channels * 2, out_channels)
        self.lin_passing = torch.nn.Linear(out_channels + in_channels, out_channels)
        self.batch_norm = BatchNorm(out_channels)

    def forward(self, x, edge_index, batch):
        x_transformed = self.lin_node(x)
        return self.propagate(edge_index, x=x_transformed, original_x=x, batch=batch)

    def message(self, edge_index_i, x_i, x_j):
        x = torch.cat([x_i, x_j], dim=1)
        m = self.lin_message(x)
        return m

    def update(self, aggr_out, edge_index, x, original_x, batch):
        n_graphs = torch.unique(batch).shape[0]
        cross_graph_attention, a_x, a_y = batch_block_pair_attention(
            original_x, batch, n_graphs
        )
        attention_input = original_x - cross_graph_attention
        aggr_out = self.lin_passing(torch.cat([aggr_out, attention_input], dim=1))
        aggr_out = self.batch_norm(aggr_out)

        norms = torch.norm(aggr_out, p=2, dim=1)
        cross_attention_sums = cross_graph_attention.sum(dim=1)

        return (
            aggr_out,
            edge_index,
            batch,
            (
                attention_input,
                cross_graph_attention,
                a_x,
                a_y,
                norms,
                cross_attention_sums,
            ),
        )


class GraphAggregator(torch.nn.Module):
    def __init__(self, in_channels, out_channels, args):
        super(GraphAggregator, self).__init__()
        self.lin = torch.nn.Linear(in_channels, out_channels)
        self.lin_gate = torch.nn.Linear(in_channels, out_channels)
        self.lin_final = torch.nn.Linear(out_channels, out_channels)
        self.args = args

    def forward(self, x, edge_index, batch):
        x_states = self.lin(x)
        x_gates = torch.nn.functional.softmax(self.lin_gate(x), dim=1)
        x_states = x_states * x_gates
        x_states = scatter_mean(x_states, batch, dim=0)
        x_states = self.lin_final(x_states)
        return x_states


class GraphMatchingNetwork(torch.nn.Module):
    def __init__(self, args):
        super(GraphMatchingNetwork, self).__init__()
        self.args = args
        self.margin = self.args.margin
        if args.n_classes > 2:
            self.f1_average = "micro"
        else:
            self.f1_average = "binary"
        self.layers = torch.nn.ModuleList()
        self.layers.append(
            GraphMatchingConvolution(self.args.feat_dim, self.args.dim, args)
        )
        for _ in range(self.args.num_layers - 1):
            self.layers.append(
                GraphMatchingConvolution(self.args.dim, self.args.dim, args)
            )
        self.aggregator = GraphAggregator(self.args.dim, self.args.dim, self.args)
        self.layer_outputs = []
        self.layer_cross_attentions = []
        self.mincut = []
        self.mlp = torch.nn.Sequential()
        self.args.n_clusters = args.n_clusters
        self.mlp.append(Linear(self.args.dim, self.args.n_clusters))
        self.topk_outputs = []
        self.norms_per_layer = []
        self.attention_sums_per_layer = []

    def compute_emb(
        self, feats, edge_index, batch, sizes_1, sizes_2, edge_index_1, edge_index_2
    ):

        topk_pooling = TopKPooling(
            self.args.dim, ratio=min(sizes_1.item(), sizes_2.item())
        )
        for i in range(self.args.num_layers):
            (
                feats,
                edge_index,
                batch,
                (
                    attention_input,
                    cross_graph_attention,
                    a_x,
                    a_y,
                    norms,
                    attention_sums,
                ),
            ) = self.layers[i](feats, edge_index, batch)

            x_1 = feats[: sizes_1.item(), :]
            x_2 = feats[sizes_1.item() : sizes_1.item() + sizes_2.item(), :]

            norms_1 = norms[: sizes_1.item()]
            norms_2 = norms[sizes_1.item() : sizes_1.item() + sizes_2.item()]

            attention_sums_1 = attention_sums[: sizes_1.item()]
            attention_sums_2 = attention_sums[
                sizes_1.item() : sizes_1.item() + sizes_2.item()
            ]

            x_pooled_1, edge_index_pooled_1, _, _, perm1, score1 = topk_pooling(
                x_1,
                edge_index_1,
            )
            x_pooled_2, edge_index_pooled_2, _, _, perm2, score2 = topk_pooling(
                x_2,
                edge_index_2,
            )

            self.topk_outputs.append(
                (
                    (x_pooled_1, edge_index_pooled_1, perm1, score1),
                    (x_pooled_2, edge_index_pooled_2, perm2, score2),
                )
            )
            self.layer_cross_attentions.append((cross_graph_attention, a_x, a_y))
            self.layer_outputs.append((x_1, edge_index_1, x_2, edge_index_2))
            self.norms_per_layer.append((norms_1, norms_2))
            self.attention_sums_per_layer.append((attention_sums_1, attention_sums_2))

        feats = self.aggregator(feats, edge_index, batch)
        return feats, edge_index, batch

    def combine_pair_embedding(
        self, feats_1, edge_index_1, feats_2, edge_index_2, sizes_1, sizes_2
    ):
        feats = torch.cat([feats_1, feats_2], dim=0)
        max_node_idx_1 = sizes_1.sum()
        edge_index_2_offset = edge_index_2 + max_node_idx_1
        edge_index = torch.cat([edge_index_1, edge_index_2_offset], dim=1)
        batch = create_batch(torch.cat([sizes_1, sizes_2], dim=0))
        feats, edge_index, batch = (
            feats.to(self.args.device),
            edge_index.to(self.args.device),
            batch.to(self.args.device),
        )
        return feats, edge_index, batch

    def forward(self, feats_1, edge_index_1, feats_2, edge_index_2, sizes_1, sizes_2):
        self.layer_outputs = []
        self.layer_cross_attentions = []
        self.topk_outputs = []
        self.mincut = []
        self.norms_per_layer = []
        self.attention_sums_per_layer = []

        feats, edge_index, batch = self.combine_pair_embedding(
            feats_1, edge_index_1, feats_2, edge_index_2, sizes_1, sizes_2
        )
        emb, _, _ = self.compute_emb(
            feats, edge_index, batch, sizes_1, sizes_2, edge_index_1, edge_index_2
        )
        emb_1 = emb[: emb.shape[0] // 2, :]
        emb_2 = emb[emb.shape[0] // 2 :, :]

        best_acc1, best_acc2 = 0.0, 0.0
        cluster1, cluster2 = None, None
        layer1, layer2 = None, None
        clusters = []

        return emb_1, emb_2, cluster1, cluster2, layer1, layer2

    def compute_metrics(self, emb_1, emb_2, labels):
        distances = torch.norm(emb_1 - emb_2, p=2, dim=1)
        loss = F.relu(self.margin - labels * (1 - distances)).mean()
        predicted_similar = torch.where(
            distances < self.args.margin,
            torch.ones_like(labels),
            -torch.ones_like(labels),
        )
        acc = (predicted_similar == labels).float().mean()
        metrics = {"loss": loss, "acc": acc}
        return metrics

    def init_metric_dict(self):
        return {"acc": -1, "f1": -1}

    def has_improved(self, m1, m2):
        return m1["acc"] < m2["acc"]

In [ ]:
# dataset = TUDataset(
#     root="data", name="MUTAG", use_node_attr=True, transform=NormalizeFeatures()
# )

dataset = GraphDataset(torch.load("data/cycle_line_star_complete_1_25+.pt"))


(
    small_graphs,
    medium_graphs,
    large_graphs,
    classes,
    small_classes,
    medium_classes,
    large_classes,
) = analyze_dataset(dataset)

In [ ]:
def train(model, optimizer, pairs, labels, batch_size, title):
    model.train()
    train_losses = []
    train_accuracies = []
    losses = []
    accs = []

    for i in range(len(pairs)):
        optimizer.zero_grad()

        graph1, graph2 = pairs[i]
        label = labels[i]

        feats_1, edge_index_1 = graph1.x, graph1.edge_index
        feats_2, edge_index_2 = graph2.x, graph2.edge_index
        sizes_1 = torch.tensor([graph1.num_nodes])
        sizes_2 = torch.tensor([graph2.num_nodes])

        emb_1, emb_2, _, _, _, _ = model(
            feats_1, edge_index_1, feats_2, edge_index_2, sizes_1, sizes_2
        )

        metrics = model.compute_metrics(emb_1, emb_2, torch.tensor([label]))
        loss = metrics["loss"]
        acc = metrics["acc"]

        losses.append(loss)
        accs.append(acc)

        if i % batch_size == 0 and i > 0:
            batch_loss = torch.mean(torch.stack(losses))
            batch_acc = torch.mean(torch.stack(accs))
            losses = []
            accs = []
            train_losses.append(batch_loss.detach().numpy())
            train_accuracies.append(batch_acc.detach().numpy())
            batch_loss.backward()
            optimizer.step()

    plt.figure(figsize=(12, 5))
    plt.title(title)

    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label="Training Loss")
    plt.title("Loss over Epochs")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(train_accuracies, label="Training Accuracy")
    plt.title("Accuracy over Epochs")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend()

    plt.show()

In [ ]:
class NewArgs:
    def __init__(self, dim, num_layers, margin, lr, batch_size, num_pairs):
        self.dim = dim
        self.feat_dim = dataset.num_features
        self.num_layers = num_layers
        self.margin = margin
        self.lr = lr
        self.n_classes = dataset.num_classes
        self.batch_size = batch_size
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.n_clusters = 8
        self.num_pairs = num_pairs

In [ ]:
top_10 = [
    (32, 7, 0.2, 0.01, 64, 500),
    (32, 5, 0.2, 0.01, 32, 3000),
    (32, 7, 0.5, 0.01, 32, 3000),
    (32, 5, 0.1, 0.01, 64, 3000),
    (32, 7, 0.5, 0.01, 128, 1000),
]

In [ ]:
# for hyperparams in top_10:
#     newargs = NewArgs(*hyperparams)
#     model = GraphMatchingNetwork(newargs)
#     optimizer = Adam(model.parameters(), lr=newargs.lr, weight_decay=1e-5)
#     pairs, labels = create_graph_pairs(dataset, newargs.num_pairs)
#     train(model, optimizer, pairs, labels, newargs.batch_size, str(hyperparams))

In [ ]:
def extract_embeddings_and_attention(
    model, feats_1, edge_index_1, feats_2, edge_index_2, sizes_1, sizes_2
):
    model(feats_1, edge_index_1, feats_2, edge_index_2, sizes_1, sizes_2)
    embeddings = model.layer_outputs
    attentions = model.layer_cross_attentions
    return embeddings, attentions


def extract_dynamic_attention_nodes(attentions, threshold=0.1):
    attention_nodes = []
    softmax = torch.nn.Softmax(dim=1)

    for layer_idx, (cross_graph_attention, a_x, a_y) in enumerate(attentions):
        dynamic_attention_nodes_1_to_2 = []
        dynamic_attention_nodes_2_to_1 = []

        a_x = a_x[0]
        a_y = a_y[0]

        assert a_x.shape[0] == a_y.shape[0]
        assert a_x.shape[1] == a_y.shape[1]

        # # Apply softmax normalization
        # a_x = softmax(a_x)
        # a_y = softmax(a_y.T).T

        for i, x_attention in enumerate(a_x):
            strong_attention_indices = (x_attention > threshold).nonzero(as_tuple=True)[
                0
            ]
            dynamic_attention_nodes_1_to_2.append(strong_attention_indices.tolist())

        for j, y_attention in enumerate(a_y.T):
            strong_attention_indices = (y_attention > threshold).nonzero(as_tuple=True)[
                0
            ]
            dynamic_attention_nodes_2_to_1.append(strong_attention_indices.tolist())

        attention_nodes.append(
            (dynamic_attention_nodes_1_to_2, dynamic_attention_nodes_2_to_1)
        )

    for i in range(len(attention_nodes)):
        for j in range(i):
            if attention_nodes[i] == attention_nodes[j]:
                for k in range(len(attention_nodes[i][0])):
                    attention_nodes[i][0][k] = []
                for k in range(len(attention_nodes[i][1])):
                    attention_nodes[i][1][k] = []
                break

    return attention_nodes


def print_attentions(attention_nodes, d):
    n_layers = len(attention_nodes)
    max_length = max(
        max(len(attn_1_to_2), len(attn_2_to_1))
        for attn_1_to_2, attn_2_to_1 in attention_nodes
    )

    col_width = 30

    if d == "12":
        print(" Graph 1 to Graph 2:".ljust(73))
    else:
        print(" Graph 2 to Graph 1:".ljust(73))
    print("=" * 130)

    for node_idx in range(max_length):
        line = []

        for layer_idx in range(n_layers):
            if d == "12":
                if node_idx < len(attention_nodes[layer_idx][0]):
                    attn_1_to_2 = attention_nodes[layer_idx][0][node_idx]
                    attn_1_to_2_str = f"{node_idx}: {attn_1_to_2}".ljust(col_width)
                else:
                    attn_1_to_2_str = "".ljust(col_width)

                line.append(attn_1_to_2_str)

            else:
                if node_idx < len(attention_nodes[layer_idx][1]):
                    attn_2_to_1 = attention_nodes[layer_idx][1][node_idx]
                    attn_2_to_1_str = f"{node_idx}: {attn_2_to_1}".ljust(col_width)
                else:
                    attn_2_to_1_str = "".ljust(col_width)

                line.append(attn_2_to_1_str)

        g1_to_g2_str = " | ".join(line)

        print(f"{g1_to_g2_str}")

    print("=" * 130)

In [ ]:
# c = f"class_0"
# idx1 = random.sample(range(len(small_classes[c])), 1)[0]
# idx2 = random.sample(range(len(small_classes[c])), 1)[0]
# graph1, graph2 = small_classes[c][idx1], small_classes[c][idx2]
# model.eval()

# feats_1, edge_index_1 = graph1.x, graph1.edge_index
# feats_2, edge_index_2 = graph2.x, graph2.edge_index
# sizes_1 = torch.tensor([len(graph1.x)])
# sizes_2 = torch.tensor([len(graph2.x)])
# emb1, emb2, cluster1, cluster2, layer1, layer2 = model(
#     feats_1, edge_index_1, feats_2, edge_index_2, sizes_1, sizes_2
# )

# embeddings, attentions = extract_embeddings_and_attention(
#     model, feats_1, edge_index_1, feats_2, edge_index_2, sizes_1, sizes_2
# )

# attention_nodes = extract_dynamic_attention_nodes(attentions, threshold=0.1)

In [ ]:
(graph1, graph2, attention_nodes) = torch.load("info.pt")

In [ ]:
def mutual_pairs(attention_nodes, i=0):
    outer_layer = attention_nodes[i]
    g1_attention, g2_attention = outer_layer

    mutual_pairs = []

    for g1_node, g1_attends in enumerate(g1_attention):
        for g2_node in g1_attends:
            if g1_node in g2_attention[g2_node]:
                pair = (g1_node, g2_node)
                if pair not in mutual_pairs:
                    mutual_pairs.append(pair)

    random.shuffle(mutual_pairs)
    return mutual_pairs


# pairs = mutual_pairs(attention_nodes, 1)
# pairs.sort()
# reversed_pairs = [(p[1], p[0]) for p in pairs]
# reversed_pairs.sort()
# print(reversed_pairs)

In [ ]:
# def explore(graph1, graph2, attention_nodes, to_print=True):
#     explored = set()
#     patterns = set()

#     def find_all_mutual_pairs(g1_frontier, g2_frontier, mp, explored):
#         g1_frontier_list = list(g1_frontier)
#         random.shuffle(g1_frontier_list)

#         g2_frontier_list = list(g2_frontier)
#         random.shuffle(g2_frontier_list)
#         mutual_pairs_set = set(mp)
#         pairs = []

#         for node_g1 in g1_frontier_list:
#             for node_g2 in g2_frontier_list:
#                 if (node_g1, node_g2) in mutual_pairs_set:
#                     if not (node_g1, node_g2) in explored:
#                         pairs.append((node_g1, node_g2))

#         return pairs

#     def step(n1, n2, g1_frontier, g2_frontier, g1_explored, g2_explored, pattern, mp):
#         for i in find_neighbors(graph1, n1):
#             if not i in g1_explored:
#                 g1_frontier.add(i)

#         for i in find_neighbors(graph2, n2):
#             if not i in g2_explored:
#                 g2_frontier.add(i)

#         # print(g1_frontier, g2_frontier)
#         all_mp = find_all_mutual_pairs(g1_frontier, g2_frontier, mp, explored)
#         pattern.add((n1, n2))
#         # print(all_mp)

#         if all_mp == [] or (
#             len(g1_frontier) == len(graph1.x) or len(g2_frontier) == len(graph2.x)
#         ):
#             if len(pattern) > 2:
#                 patterns.add(frozenset(pattern))
#             # print("---------")
#             return
#         else:
#             for p in all_mp:
#                 n1, n2 = p
#                 # print(g1_frontier, g2_frontier)
#                 # print(p)
#                 # print("---------")

#                 g1_frontier_copy = g1_frontier.copy()
#                 g2_frontier_copy = g2_frontier.copy()
#                 g1_explored_copy = g1_explored.copy()
#                 g2_explored_copy = g2_explored.copy()

#                 g1_frontier_copy.remove(n1)
#                 g2_frontier_copy.remove(n2)
#                 g1_explored_copy.add(n1)
#                 g2_explored_copy.add(n2)
#                 step(
#                     n1,
#                     n2,
#                     g1_frontier_copy.copy(),
#                     g2_frontier_copy.copy(),
#                     g1_explored_copy.copy(),
#                     g2_explored_copy.copy(),
#                     pattern.copy(),
#                     mp,
#                 )

#     def find_neighbors(graph, node):
#         neighbors = graph.edge_index[1, graph.edge_index[0] == node].tolist()
#         neighbors += graph.edge_index[0, graph.edge_index[1] == node].tolist()
#         neighbors = list(set(neighbors))
#         return neighbors

#     for k in range(len(attention_nodes)):

#         # print(f"Layer {k}")
#         # print("======================")

#         pairs = mutual_pairs(attention_nodes, k)

#         for pair in pairs:
#             if pair in explored:
#                 continue

#             n1, n2 = pair

#             # print(f"Starting with: {pair}")
#             # print("---")

#             g1_explored, g2_explored = set(), set()
#             g1_frontier, g2_frontier = set(), set()
#             g1_explored.add(n1)
#             g2_explored.add(n2)
#             pattern = set()

#             step(
#                 n1,
#                 n2,
#                 g1_frontier.copy(),
#                 g2_frontier.copy(),
#                 g1_explored.copy(),
#                 g2_explored.copy(),
#                 pattern.copy(),
#                 pairs,
#             )

#         for pat in patterns:
#             for pair_in_pattern in pat:
#                 explored.add(pair_in_pattern)

#     return patterns


# patterns = explore(graph1, graph2, attention_nodes, False)

In [ ]:
def create_subgraphs(pattern, graph1, graph2):
    g1_nodes = set()
    g2_nodes = set()
    for g1_node, g2_node in pattern.items():
        g1_nodes.add(g1_node)
        g2_nodes.add(g2_node)

    g1_node_map = {node: idx for idx, node in enumerate(g1_nodes)}
    g2_node_map = {node: idx for idx, node in enumerate(g2_nodes)}

    g1_edge_index = []
    for edge in graph1.edge_index.t():
        if edge[0].item() in g1_nodes and edge[1].item() in g1_nodes:
            g1_edge_index.append(
                [g1_node_map[edge[0].item()], g1_node_map[edge[1].item()]]
            )

    g2_edge_index = []
    for edge in graph2.edge_index.t():
        if edge[0].item() in g2_nodes and edge[1].item() in g2_nodes:
            g2_edge_index.append(
                [g2_node_map[edge[0].item()], g2_node_map[edge[1].item()]]
            )

    g1_subgraph = Data(
        x=graph1.x[list(g1_nodes)],
        edge_index=torch.tensor(g1_edge_index, dtype=torch.long).t().contiguous(),
        original_node_ids=torch.tensor(list(g1_nodes), dtype=torch.long),
    )
    g2_subgraph = Data(
        x=graph2.x[list(g2_nodes)],
        edge_index=torch.tensor(g2_edge_index, dtype=torch.long).t().contiguous(),
        original_node_ids=torch.tensor(list(g2_nodes), dtype=torch.long),
    )

    return g1_subgraph, g2_subgraph


def mutual_pairs(attention_nodes, i=0):
    outer_layer = attention_nodes[i]
    g1_attention, g2_attention = outer_layer

    mutual_pairs = []

    for g1_node, g1_attends in enumerate(g1_attention):
        for g2_node in g1_attends:
            if g1_node in g2_attention[g2_node]:
                pair = (g1_node, g2_node)
                if pair not in mutual_pairs:
                    mutual_pairs.append(pair)

    random.shuffle(mutual_pairs)
    return mutual_pairs


def print_patterns(patterns, graph1, graph2):
    for i in patterns:
        g1_subgraph, g2_subgraph = create_subgraphs(i, graph1, graph2)

        if nx.is_isomorphic(
            to_networkx(g1_subgraph, to_undirected=True),
            to_networkx(g2_subgraph, to_undirected=True),
        ):

            plot_mutag(
                g1_subgraph,
                g2_subgraph,
                perm1=g1_subgraph.original_node_ids,
                perm2=g2_subgraph.original_node_ids,
            )

In [ ]:
import networkx as nx
import torch
from torch_geometric.utils import degree
import time


class MCS:
    def __init__(self, mp):
        self.mp = mp
        self.max_size = 0
        self.all_mappings = []
        self.unique_mappings = set()
        self.visited = set()
        self.time_limit = 15
        self.start_time = None

    def find_mcs(self, G1, G2):
        self.max_size = 0
        self.all_mappings = []
        self.unique_mappings = set()
        self.visited = set()
        self.start_time = time.time()

        G1_degrees = degree(G1.edge_index[0], G1.num_nodes)
        G2_degrees = degree(G2.edge_index[0], G2.num_nodes)

        nodes1 = list(range(G1.num_nodes))
        nodes2 = list(range(G2.num_nodes))

        for n1 in nodes1:
            for n2 in nodes2:
                if (n1, n2) in self.mp:
                    M = {n1: n2}
                    neighbors1 = set(G1.edge_index[1][G1.edge_index[0] == n1].tolist())
                    neighbors2 = set(G2.edge_index[1][G2.edge_index[0] == n2].tolist())
                    self.match(
                        G1, G2, M, G1_degrees, G2_degrees, neighbors1, neighbors2
                    )
        return self.all_mappings

    def match(self, G1, G2, M, G1_degrees, G2_degrees, neighbors1, neighbors2):
        if time.time() - self.start_time > self.time_limit:
            return

        state = (frozenset(M.items()), frozenset(neighbors1), frozenset(neighbors2))
        if state in self.visited:
            return
        self.visited.add(state)

        if len(M) > self.max_size:
            self.max_size = len(M)
            self.all_mappings = [M.copy()]
            self.unique_mappings = {self.canonical_form(M)}
        # elif len(M) == self.max_size:
        #     canonical = self.canonical_form(M)
        #     if canonical not in self.unique_mappings:
        #         self.all_mappings.append(M.copy())
        #         self.unique_mappings.add(canonical)

        candidates1 = sorted(neighbors1, key=lambda n: -G1_degrees[n].item())
        candidates2 = sorted(neighbors2, key=lambda n: -G2_degrees[n].item())

        for n1 in candidates1:
            if n1 not in M:
                for n2 in candidates2:
                    if n2 not in M.values() and self.feasible(n1, n2, M, G1, G2):
                        M[n1] = n2
                        new_neighbors1 = set(
                            G1.edge_index[1][G1.edge_index[0] == n1].tolist()
                        )
                        new_neighbors2 = set(
                            G2.edge_index[1][G2.edge_index[0] == n2].tolist()
                        )
                        neighbors1.update(new_neighbors1 - set(M.keys()))
                        neighbors2.update(new_neighbors2 - set(M.values()))
                        self.match(
                            G1,
                            G2,
                            M,
                            G1_degrees,
                            G2_degrees,
                            neighbors1,
                            neighbors2,
                        )
                        del M[n1]
                        neighbors1.difference_update(new_neighbors1)
                        neighbors2.difference_update(new_neighbors2)

    def feasible(self, n1, n2, M, G1, G2):
        if not torch.equal(G1.x[n1], G2.x[n2]):
            return False
        if (n1, n2) not in self.mp:
            return False

        count1 = 0
        count2 = 0

        for neighbor in G1.edge_index[1][G1.edge_index[0] == n1]:
            if neighbor.item() in M:
                count1 += 1

        for neighbor in G2.edge_index[1][G2.edge_index[0] == n2]:
            if neighbor.item() in M.values():
                count2 += 1

        if count1 != count2:
            return False

        for neighbor in G1.edge_index[1][G1.edge_index[0] == n1]:
            if (
                neighbor.item() in M
                and not (
                    G2.edge_index[1][G2.edge_index[0] == n2] == M[neighbor.item()]
                ).any()
            ):
                return False

        return True

    def canonical_form(self, M):
        G1_set = set(M.keys())
        G2_set = set(M.values())
        return (frozenset(G1_set), frozenset(G2_set))

In [ ]:
hyperparams = (32, 7, 0.2, 0.01, 64, 500)
newargs = NewArgs(*hyperparams)
model = GraphMatchingNetwork(newargs)
optimizer = Adam(model.parameters(), lr=newargs.lr, weight_decay=1e-5)
pairs, labels = create_graph_pairs(dataset, newargs.num_pairs)
train(model, optimizer, pairs, labels, newargs.batch_size, str(hyperparams))

In [ ]:
total = 0
correct_class0 = 0
correct_class1 = 0
correct_class2 = 0
correct_class3 = 0
correct_class0_exact = 0
correct_class1_exact = 0
correct_class2_exact = 0
correct_class3_exact = 0
ts = []
for cn in range(4):
    for j in range(25):
        c = f"class_{cn}"
        print(f"Class {cn} - Pair {j}")
        idx1 = random.sample(range(len(classes[c])), 1)[0]
        idx2 = random.sample(range(len(classes[c])), 1)[0]
        graph1, graph2 = classes[c][idx1], classes[c][idx2]
        model.eval()

        feats_1, edge_index_1 = graph1.x, graph1.edge_index
        feats_2, edge_index_2 = graph2.x, graph2.edge_index
        sizes_1 = torch.tensor([len(graph1.x)])
        sizes_2 = torch.tensor([len(graph2.x)])
        emb1, emb2, cluster1, cluster2, layer1, layer2 = model(
            feats_1, edge_index_1, feats_2, edge_index_2, sizes_1, sizes_2
        )

        embeddings, attentions = extract_embeddings_and_attention(
            model, feats_1, edge_index_1, feats_2, edge_index_2, sizes_1, sizes_2
        )

        br = False
        for i in range(7):
            # for t in [0.05, 0.08, 0.09, 0.1, 0.11, 0.12]:
            for t in [0.02, 0.03, 0.04, 0.05]:
                attention_nodes = extract_dynamic_attention_nodes(
                    attentions, threshold=t
                )
                mp = mutual_pairs(attention_nodes, i)
                vf2 = MCS(mp)
                pattern = vf2.find_mcs(graph1, graph2)
                if pattern != [] and len(pattern[0]) >= 23:
                    g1_subgraph, g2_subgraph = create_subgraphs(
                        pattern[0], graph1, graph2
                    )

                    if (
                        nx.is_isomorphic(
                            to_networkx(g1_subgraph, to_undirected=True),
                            to_networkx(g2_subgraph, to_undirected=True),
                        )
                        and len(pattern[0]) > 2
                    ):
                        summary = Data(
                            x=(g1_subgraph.x + g2_subgraph.x) / 2,
                            edge_index=g1_subgraph.edge_index,
                        )

                        # plot_graph(summary)

                        if c == "class_0":
                            correct_class0 += is_cycle(summary)
                            if is_cycle(summary):
                                if len(pattern[0]) == 25:
                                    correct_class0_exact += 1
                                br = True
                                ts.append(t)
                                break
                        elif c == "class_1":
                            correct_class1 += is_complete(summary)
                            if is_complete(summary):
                                if len(pattern[0]) == 25:
                                    correct_class1_exact += 1
                                ts.append(t)
                                br = True
                                break
                        elif c == "class_2":
                            correct_class2 += is_line(summary)
                            if is_line(summary):
                                if len(pattern[0]) == 25:
                                    correct_class2_exact += 1
                                br = True
                                ts.append(t)
                                break
                        elif c == "class_3":
                            correct_class3 += is_star(summary)
                            if is_star(summary):
                                if len(pattern[0]) == 25:
                                    correct_class3_exact += 1
                                br = True
                                ts.append(t)
                                break

            if br:
                break

    print(
        f"{correct_class0} ({correct_class0_exact})",
        f"{correct_class1} ({correct_class1_exact})",
        f"{correct_class2} ({correct_class2_exact})",
        f"{correct_class3} ({correct_class3_exact})",
        f"{correct_class0 + correct_class1 + correct_class2 + correct_class3} ({correct_class0_exact + correct_class1_exact + correct_class2_exact + correct_class3_exact})",
    )
    print(Counter(ts))

In [ ]:
c = f"class_{cn}"
print(f"Class {cn} - Pair {j}")
idx1 = random.sample(range(len(classes[c])), 1)[0]
idx2 = random.sample(range(len(classes[c])), 1)[0]
graph1, graph2 = classes[c][idx1], classes[c][idx2]
model.eval()

feats_1, edge_index_1 = graph1.x, graph1.edge_index
feats_2, edge_index_2 = graph2.x, graph2.edge_index
sizes_1 = torch.tensor([len(graph1.x)])
sizes_2 = torch.tensor([len(graph2.x)])
emb1, emb2, cluster1, cluster2, layer1, layer2 = model(
    feats_1, edge_index_1, feats_2, edge_index_2, sizes_1, sizes_2
)

embeddings, attentions = extract_embeddings_and_attention(
    model, feats_1, edge_index_1, feats_2, edge_index_2, sizes_1, sizes_2
)

br = False
for i in range(7):
    for t in [0.02]:
        attention_nodes = extract_dynamic_attention_nodes(attentions, threshold=t)
        mp = mutual_pairs(attention_nodes, i)
        vf2 = MCS(mp)
        pattern = vf2.find_mcs(graph1, graph2)
        if pattern != [] and len(pattern[0]) >= 23:
            g1_subgraph, g2_subgraph = create_subgraphs(pattern[0], graph1, graph2)

            if (
                nx.is_isomorphic(
                    to_networkx(g1_subgraph, to_undirected=True),
                    to_networkx(g2_subgraph, to_undirected=True),
                )
                and len(pattern[0]) > 2
            ):
                summary = Data(
                    x=(g1_subgraph.x + g2_subgraph.x) / 2,
                    edge_index=g1_subgraph.edge_index,
                )

                plot_graph(summary)

In [ ]:
# 8

# 7or8: 21 20 20 20 81

# 8:    18 18 18 13 67

# ---------------

# 15

# 15: 15 20 17 9 61

# ---------------

# 25

# 25: 17 16 20 9 62